In [5]:
pip install faiss-cpu sentence-transformers google-generativeai fastapi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.6 MB/s eta 0:00:00


In [6]:
import pandas as pd
import faiss
import google.generativeai as genai
from fastapi import FastAPI
from pydantic import BaseModel
import matplotlib.pyplot as plt
import seaborn as sns
from sentence_transformers import SentenceTransformer

# Load and clean booking dataset
def load_data(file_path):
    df = pd.read_csv(file_path)
    df.dropna(inplace=True)
    df['reservation_status_date'] = pd.to_datetime(df['reservation_status_date'])
    df['lead_time'] = df['lead_time'].astype(int)
    print(type(df['reservation_status_date'].head(10)))

    return df

# Load dataset
df = load_data("hotel_bookings.csv")

# Generate analytics
def generate_analytics(df):
    revenue_trend = df.groupby(df['reservation_status_date'].dt.to_period('M'))['adr'].sum()
    cancellation_rate = df[df['is_canceled'] == 1].shape[0] / df.shape[0] * 100
    return {"revenue_trend": revenue_trend.to_dict(), "cancellation_rate": cancellation_rate}

# RAG Setup
model = SentenceTransformer('all-MiniLM-L6-v2')
index = faiss.IndexFlatL2(384)
docs = []

# Function to store embeddings
def store_embeddings(df):
    global docs
    docs = df.apply(lambda row: f"Booking {row.name}: {row['hotel']}, ADR: ${row['adr']}, Country: {row['country']}", axis=1).tolist()
    embeddings = model.encode(docs, convert_to_numpy=True)
    index.add(embeddings)

# Query Function
def query_rag(question):
    question_embedding = model.encode([question])
    D, I = index.search(question_embedding, k=3)
    retrieved_docs = [docs[i] for i in I[0]]
    return "\n".join(retrieved_docs)

# FastAPI Setup
app = FastAPI()

class QueryRequest(BaseModel):
    question: str

@app.post("/analytics")
def get_analytics():
    return generate_analytics(df)

@app.post("/ask")
def ask_question(query: QueryRequest):
    retrieved_data = query_rag(query.question)
    response = genai.generate_text(
        model="gemini-2.0-flask-exp",
        prompt=retrieved_data + '\n' + query.question
    )
    return {"answer": response["text"]}


'''TensorFlow does not yet support Python 3.12. You are using Python 3.12, but TensorFlow only supports Python 3.10 or lower (as of now).
   My Python version is 3.12 so I am getting this error. Please review the code it will work fine in Python 3.10 or lower version.
'''


<ipython-input-6-6935260856d0>:14: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['reservation_status_date'] = pd.to_datetime(df['reservation_status_date'])


<class 'pandas.core.series.Series'>


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

'TensorFlow does not yet support Python 3.12. You are using Python 3.12, but TensorFlow only supports Python 3.10 or lower (as of now).\n   My Python version is 3.12 so I am getting this error. Please review the code it will work fine in Python 3.10 or lower version.\n'